In [228]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [229]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [230]:
cur_folder_name = os.getcwd().split('/')[-1]
if cur_folder_name != "Virny":
    os.chdir("../..")

print('Current location: ', os.getcwd())

Current location:  /home/denys_herasymuk/UCU/4course_2term/Bachelor_Thesis/Code/Virny


# Benchmark

## Import dependencies

In [231]:
import os
import pandas as pd

from datetime import datetime, timezone
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from docs.examples.benchmark_utils import clear_directory, create_averaged_dfs_dict, populate_benchmark_report
from virny.datasets.data_loaders import CompasWithoutSensitiveAttrsDataset, ACSEmploymentDataset
from virny.user_interfaces.metrics_computation_interfaces import compute_metrics_multiple_runs
from virny.utils.custom_initializers import create_config_obj, read_model_metric_dfs

## COMPAS_Without_Sensitive_Attributes Dataset

## Initialize Input Variables

### Create a models config

In [232]:
models_config = {
    'DecisionTreeClassifier': DecisionTreeClassifier(criterion='gini',
                                                     max_depth=20,
                                                     max_features=0.6,
                                                     min_samples_split=0.1),
    'LogisticRegression': LogisticRegression(C=1,
                                             max_iter=50,
                                             penalty='l2',
                                             solver='newton-cg'),
    'RandomForestClassifier': RandomForestClassifier(max_depth=4,
                                                     max_features=0.6,
                                                     min_samples_leaf=1,
                                                     n_estimators=50),
    'XGBClassifier': XGBClassifier(learning_rate=0.1,
                                   max_depth=5,
                                   n_estimators=20),
}

### Create a Dataset class

In [233]:
dataset = CompasWithoutSensitiveAttrsDataset(dataset_path=os.path.join('virny', 'datasets', 'COMPAS.csv'))
dataset.X_data.head()

,juv_fel_count,juv_misd_count,juv_other_count,priors_count,age_cat_25 - 45,age_cat_Greater than 45,age_cat_Less than 25,c_charge_degree_F,c_charge_degree_M
0,0.0,-2.340451,1.0,-15.010999,1,0,0,0,1
1,0.0,0.000000,0.0,0.000000,1,0,0,1,0
2,0.0,0.000000,0.0,0.000000,0,0,1,1,0
3,0.0,0.000000,0.0,6.000000,1,0,0,0,1
4,0.0,0.000000,0.0,7.513697,1,0,0,1,0


### Create a config object

In [234]:
ROOT_DIR = os.path.join('docs', 'examples')
config_yaml_path = os.path.join(ROOT_DIR, 'experiment_config.yaml')
config_yaml_content = """
dataset_name: COMPAS_Without_Sensitive_Attributes
test_set_fraction: 0.2
bootstrap_fraction: 0.8
n_estimators: 100
runs_seed_lst: [100, 200, 300]
#runs_seed_lst: [100, 200]
sensitive_attributes_dct: {'sex': 0, 'race': 'Caucasian', 'sex&race': None}
"""

with open(config_yaml_path, 'w', encoding='utf-8') as f:
    f.write(config_yaml_content)

In [235]:
config = create_config_obj(config_yaml_path=config_yaml_path)
SAVE_RESULTS_DIR_PATH1 = os.path.join(ROOT_DIR, 'results', 'benchmark', 'benchmark_results', config.dataset_name)
STANDARD_RESULTS_DIR_PATH1 = os.path.join(ROOT_DIR, 'results', 'benchmark', 'standard_results', config.dataset_name)

In [236]:
clear_directory(SAVE_RESULTS_DIR_PATH1)

Directory is cleared


## Subgroup Metrics Computation

In [237]:
multiple_run_metrics_dct = compute_metrics_multiple_runs(dataset, config, models_config, SAVE_RESULTS_DIR_PATH1, verbose=1)

Multiple runs progress:   0%|          | 0/3 [00:00<?, ?it/s]

Analyze models in one run:   0%|          | 0/4 [00:00<?, ?it/s]

##############################  [Model 1 / 4] Analyze DecisionTreeClassifier  ##############################
Model random_state:  101
Baseline X_train shape:  (4222, 9)
Baseline X_test shape:  (1056, 9)




2023-02-04 15:21:53 abstract_overall_variance_analyzer.py INFO    : Start classifiers testing by bootstrap


Classifiers testing by bootstrap:   0%|          | 0/100 [00:00<?, ?it/s]

...

2023-02-04 15:24:08 abstract_overall_variance_analyzer.py INFO    : Successfully tested classifiers by bootstrap
2023-02-04 15:24:10 abstract_overall_variance_analyzer.py INFO    : Successfully computed predict proba metrics






##############################  [Model 4 / 4] Analyze XGBClassifier  ##############################
Model random_state:  304
Baseline X_train shape:  (4222, 9)
Baseline X_test shape:  (1056, 9)




2023-02-04 15:24:17 abstract_overall_variance_analyzer.py INFO    : Start classifiers testing by bootstrap


Classifiers testing by bootstrap:   0%|          | 0/100 [00:00<?, ?it/s]

2023-02-04 15:24:22 abstract_overall_variance_analyzer.py INFO    : Successfully tested classifiers by bootstrap
2023-02-04 15:24:25 abstract_overall_variance_analyzer.py INFO    : Successfully computed predict proba metrics


In [238]:
sample_model_metrics_df = multiple_run_metrics_dct[list(models_config.keys())[0]]
sample_model_metrics_df.head(20)

,Metric,overall,sex_priv,sex_dis,race_priv,race_dis,sex&race_priv,sex&race_dis,Model_Seed,Model_Name,Run_Number
0,Mean,0.527155,0.559299,0.519691,0.592431,0.482842,0.590330,0.471730,101,DecisionTreeClassifier,Run_1
1,Std,0.072303,0.077301,0.071142,0.070465,0.073550,0.087271,0.074440,101,DecisionTreeClassifier,Run_1
2,IQR,0.091349,0.097852,0.089839,0.092370,0.090656,0.112774,0.091649,101,DecisionTreeClassifier,Run_1
3,Entropy,0.000000,0.276445,0.000000,0.000000,0.268387,0.267615,0.265161,101,DecisionTreeClassifier,Run_1
4,Jitter,0.162999,0.183252,0.158296,0.143923,0.175948,0.177904,0.173475,101,DecisionTreeClassifier,Run_1
5,Per_Sample_Accuracy,0.653400,0.663618,0.651027,0.656370,0.651383,0.643750,0.645386,101,DecisionTreeClassifier,Run_1
6,Label_Stability,0.759943,0.724221,0.768238,0.781499,0.745310,0.728409,0.750541,101,DecisionTreeClassifier,Run_1
7,TPR,0.593429,0.542857,0.601918,0.427673,0.673780,0.440000,0.685512,101,DecisionTreeClassifier,Run_1
8,TNR,0.743409,0.759690,0.738636,0.805970,0.687708,0.730159,0.659574,101,DecisionTreeClassifier,Run_1
9,PPV,0.664368,0.550725,0.685792,0.566667,0.701587,0.392857,0.708029,101,DecisionTreeClassifier,Run_1


## Create a Benchmark Report

In [239]:
models_metrics_dct = read_model_metric_dfs(SAVE_RESULTS_DIR_PATH1, model_names=models_config.keys())
standard_models_metrics_dct = read_model_metric_dfs(STANDARD_RESULTS_DIR_PATH1, model_names=models_config.keys())

avg_models_metrics_dct = create_averaged_dfs_dict(models_metrics_dct)
avg_standard_models_metrics_dct = create_averaged_dfs_dict(standard_models_metrics_dct)

In [240]:
report_df = pd.DataFrame()
report_df = populate_benchmark_report(report_df, avg_models_metrics_dct, avg_standard_models_metrics_dct,
                                      config.dataset_name, config.sensitive_attributes_dct)

In [241]:
report_df

,Dataset,Model,Subgroup,Delta%_Accuracy,Delta%_Entropy,Delta%_F1,Delta%_FNR,Delta%_FPR,Delta%_IQR,Delta%_Jitter,Delta%_Label_Stability,Delta%_Mean,Delta%_PPV,Delta%_Per_Sample_Accuracy,Delta%_Positive-Rate,Delta%_Selection-Rate,Delta%_Std,Delta%_TNR,Delta%_TPR
0,COMPAS_Without_Sensitive_Attributes,DecisionTreeClassifier,overall,0.095,0.775,0.146,-0.201,0.008,0.016,0.676,-1.284,0.049,0.076,-0.145,0.192,0.095,0.124,-0.008,0.201
1,COMPAS_Without_Sensitive_Attributes,DecisionTreeClassifier,sex_priv,0.006,0.538,0.019,0.000,0.000,-0.021,0.479,-1.167,0.108,0.015,0.035,-0.049,-0.006,0.116,0.000,0.000
2,COMPAS_Without_Sensitive_Attributes,DecisionTreeClassifier,sex_dis,0.117,0.829,0.167,-0.236,0.009,0.024,0.723,-1.312,0.035,0.079,-0.187,0.228,0.117,0.126,-0.009,0.236
3,COMPAS_Without_Sensitive_Attributes,DecisionTreeClassifier,race_priv,-0.002,0.496,0.125,-0.210,0.134,0.016,0.537,-1.032,0.041,-0.024,-0.033,0.409,0.158,0.051,-0.134,0.210
4,COMPAS_Without_Sensitive_Attributes,DecisionTreeClassifier,race_dis,0.158,1.012,0.164,-0.197,-0.101,0.018,0.769,-1.454,0.055,0.129,-0.216,0.089,0.052,0.172,0.101,0.197
5,COMPAS_Without_Sensitive_Attributes,DecisionTreeClassifier,sex&race_priv,0.000,7.311,0.000,0.000,0.000,0.125,0.504,-1.236,0.242,0.000,0.097,0.000,0.000,0.082,0.000,0.000
6,COMPAS_Without_Sensitive_Attributes,DecisionTreeClassifier,sex&race_dis,0.190,1.083,0.186,-0.227,-0.133,0.050,0.833,-1.526,0.067,0.140,-0.257,0.109,0.065,0.177,0.133,0.227
7,COMPAS_Without_Sensitive_Attributes,LogisticRegression,overall,0.063,0.061,0.231,-0.465,0.301,0.099,0.269,-0.311,-0.016,-0.096,-0.023,0.796,0.379,0.084,-0.301,0.465
8,COMPAS_Without_Sensitive_Attributes,LogisticRegression,sex_priv,0.000,0.000,0.185,-0.392,0.258,0.044,0.248,-0.379,-0.020,-0.083,-0.074,0.784,0.312,0.069,-0.258,0.392
9,COMPAS_Without_Sensitive_Attributes,LogisticRegression,sex_dis,0.077,0.067,0.236,-0.478,0.316,0.111,0.274,-0.294,-0.015,-0.105,-0.012,0.801,0.397,0.087,-0.316,0.478


## Folktables [GA, 2018] Dataset

In [242]:
dataset2 = ACSEmploymentDataset(state=['GA'], year=2018, root_dir=os.path.join('virny', 'datasets'), with_nulls=False, subsample=20_000)
dataset2.X_data.head()

,MAR,MIL,ESP,MIG,DREM,NATIVITY,DIS,DEAR,DEYE,SEX,RAC1P,RELP,CIT,ANC,SCHL,AGEP
71506,1,4,0,1,2,1,2,2,2,1,1,1,1,1,16,61
64254,3,4,0,1,2,1,2,2,2,2,2,10,1,4,21,79
96328,5,4,0,1,2,1,1,2,2,2,2,0,1,1,13,69
47767,1,2,0,1,2,1,2,2,2,1,1,0,1,1,19,67
46198,5,4,0,1,2,1,2,2,2,1,1,7,1,4,14,17


In [243]:
models_config2 = {
    'DecisionTreeClassifier': DecisionTreeClassifier(criterion='gini',
                                                     max_depth=10,
                                                     max_features=0.6,
                                                     min_samples_split=0.1),
    'LogisticRegression': LogisticRegression(C=1,
                                             max_iter=150,
                                             penalty='l2',
                                             solver='lbfgs'),
    'RandomForestClassifier': RandomForestClassifier(max_depth=10,
                                                     max_features=0.6,
                                                     min_samples_leaf=4,
                                                     n_estimators=100),
    'XGBClassifier': XGBClassifier(reg_lambda=1,
                                   learning_rate=0.1,
                                   max_depth=3,
                                   n_estimators=100),
}

In [244]:
config_yaml_path = os.path.join(ROOT_DIR, 'experiment_config.yaml')
config_yaml_content = """
dataset_name: Folktables_GA_2018
test_set_fraction: 0.2
bootstrap_fraction: 0.8
n_estimators: 100
runs_seed_lst: [100, 200, 300]
#runs_seed_lst: [100, 200, 300, 400, 500, 600]
sensitive_attributes_dct: {'SEX': '1', 'RAC1P': '1', 'SEX&RAC1P': None}
"""

with open(config_yaml_path, 'w', encoding='utf-8') as f:
    f.write(config_yaml_content)

config2 = create_config_obj(config_yaml_path=config_yaml_path)
SAVE_RESULTS_DIR_PATH2 = os.path.join(ROOT_DIR, 'results', 'benchmark', 'benchmark_results', config2.dataset_name)
STANDARD_RESULTS_DIR_PATH2 = os.path.join(ROOT_DIR, 'results', 'benchmark', 'standard_results', config2.dataset_name)

In [245]:
clear_directory(SAVE_RESULTS_DIR_PATH2)

Directory is cleared


In [246]:
multiple_run_metrics_dct2 = compute_metrics_multiple_runs(dataset2, config2, models_config2, SAVE_RESULTS_DIR_PATH2, verbose=1)

Multiple runs progress:   0%|          | 0/3 [00:00<?, ?it/s]

Analyze models in one run:   0%|          | 0/4 [00:00<?, ?it/s]

##############################  [Model 1 / 4] Analyze DecisionTreeClassifier  ##############################
Model random_state:  101
Baseline X_train shape:  (16000, 16)
Baseline X_test shape:  (4000, 16)




2023-02-04 15:24:35 abstract_overall_variance_analyzer.py INFO    : Start classifiers testing by bootstrap


Classifiers testing by bootstrap:   0%|          | 0/100 [00:00<?, ?it/s]

...

2023-02-04 15:44:36 abstract_overall_variance_analyzer.py INFO    : Successfully tested classifiers by bootstrap
2023-02-04 15:44:45 abstract_overall_variance_analyzer.py INFO    : Successfully computed predict proba metrics






##############################  [Model 4 / 4] Analyze XGBClassifier  ##############################
Model random_state:  304
Baseline X_train shape:  (16000, 16)
Baseline X_test shape:  (4000, 16)




2023-02-04 15:45:08 abstract_overall_variance_analyzer.py INFO    : Start classifiers testing by bootstrap


Classifiers testing by bootstrap:   0%|          | 0/100 [00:00<?, ?it/s]

2023-02-04 15:47:02 abstract_overall_variance_analyzer.py INFO    : Successfully tested classifiers by bootstrap
2023-02-04 15:47:12 abstract_overall_variance_analyzer.py INFO    : Successfully computed predict proba metrics


In [247]:
sample_model_metrics_df = multiple_run_metrics_dct2[list(models_config2.keys())[0]]
sample_model_metrics_df.head(20)

,Metric,overall,SEX_priv,SEX_dis,RAC1P_priv,RAC1P_dis,SEX&RAC1P_priv,SEX&RAC1P_dis,Model_Seed,Model_Name,Run_Number
0,Mean,0.553779,0.531211,0.574568,0.547523,0.566169,0.507491,0.552915,101,DecisionTreeClassifier,Run_1
1,Std,0.053819,0.057550,0.050381,0.053637,0.054179,0.056413,0.049336,101,DecisionTreeClassifier,Run_1
2,IQR,0.065751,0.070943,0.060969,0.067190,0.062902,0.071738,0.057622,101,DecisionTreeClassifier,Run_1
3,Entropy,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,101,DecisionTreeClassifier,Run_1
4,Jitter,0.049987,0.038030,0.061001,0.046342,0.057205,0.034722,0.067273,101,DecisionTreeClassifier,Run_1
5,Per_Sample_Accuracy,0.827892,0.855620,0.802349,0.829120,0.825462,0.866804,0.820314,101,DecisionTreeClassifier,Run_1
6,Label_Stability,0.928835,0.944442,0.914457,0.934612,0.917392,0.949725,0.904290,101,DecisionTreeClassifier,Run_1
7,TPR,0.866066,0.872928,0.858945,0.857734,0.883803,0.876855,0.899110,101,DecisionTreeClassifier,Run_1
8,TNR,0.810166,0.856861,0.771074,0.816425,0.798450,0.884858,0.787342,101,DecisionTreeClassifier,Run_1
9,PPV,0.784804,0.844920,0.730019,0.795856,0.762918,0.890060,0.782946,101,DecisionTreeClassifier,Run_1


In [248]:
models_metrics_dct2 = read_model_metric_dfs(SAVE_RESULTS_DIR_PATH2, model_names=models_config2.keys())
standard_models_metrics_dct2 = read_model_metric_dfs(STANDARD_RESULTS_DIR_PATH2, model_names=models_config2.keys())

avg_models_metrics_dct2 = create_averaged_dfs_dict(models_metrics_dct2)
avg_standard_models_metrics_dct2 = create_averaged_dfs_dict(standard_models_metrics_dct2)

## Create a Benchmark Report

In [249]:
report_df = populate_benchmark_report(report_df, avg_models_metrics_dct2, avg_standard_models_metrics_dct2,
                                      config2.dataset_name, config2.sensitive_attributes_dct)

In [250]:
filename = f'benchmark_report_{datetime.now(timezone.utc).strftime("%Y%m%d__%H%M%S")}'
report_df.to_csv(os.path.join(ROOT_DIR, 'results', 'benchmark', 'benchmark_reports', f'{filename}.csv'), index=False)

In [251]:
report_df

,Dataset,Model,Subgroup,Delta%_Accuracy,Delta%_Entropy,Delta%_F1,Delta%_FNR,Delta%_FPR,Delta%_IQR,Delta%_Jitter,Delta%_Label_Stability,Delta%_Mean,Delta%_PPV,Delta%_Per_Sample_Accuracy,Delta%_Positive-Rate,Delta%_Selection-Rate,Delta%_Std,Delta%_TNR,Delta%_TPR
0,COMPAS_Without_Sensitive_Attributes,DecisionTreeClassifier,overall,0.095,0.775,0.146,-0.201,0.008,0.016,0.676,-1.284,0.049,0.076,-0.145,0.192,0.095,0.124,-0.008,0.201
1,COMPAS_Without_Sensitive_Attributes,DecisionTreeClassifier,sex_priv,0.006,0.538,0.019,0.000,0.000,-0.021,0.479,-1.167,0.108,0.015,0.035,-0.049,-0.006,0.116,0.000,0.000
2,COMPAS_Without_Sensitive_Attributes,DecisionTreeClassifier,sex_dis,0.117,0.829,0.167,-0.236,0.009,0.024,0.723,-1.312,0.035,0.079,-0.187,0.228,0.117,0.126,-0.009,0.236
3,COMPAS_Without_Sensitive_Attributes,DecisionTreeClassifier,race_priv,-0.002,0.496,0.125,-0.210,0.134,0.016,0.537,-1.032,0.041,-0.024,-0.033,0.409,0.158,0.051,-0.134,0.210
4,COMPAS_Without_Sensitive_Attributes,DecisionTreeClassifier,race_dis,0.158,1.012,0.164,-0.197,-0.101,0.018,0.769,-1.454,0.055,0.129,-0.216,0.089,0.052,0.172,0.101,0.197
5,COMPAS_Without_Sensitive_Attributes,DecisionTreeClassifier,sex&race_priv,0.000,7.311,0.000,0.000,0.000,0.125,0.504,-1.236,0.242,0.000,0.097,0.000,0.000,0.082,0.000,0.000
6,COMPAS_Without_Sensitive_Attributes,DecisionTreeClassifier,sex&race_dis,0.190,1.083,0.186,-0.227,-0.133,0.050,0.833,-1.526,0.067,0.140,-0.257,0.109,0.065,0.177,0.133,0.227
7,COMPAS_Without_Sensitive_Attributes,LogisticRegression,overall,0.063,0.061,0.231,-0.465,0.301,0.099,0.269,-0.311,-0.016,-0.096,-0.023,0.796,0.379,0.084,-0.301,0.465
8,COMPAS_Without_Sensitive_Attributes,LogisticRegression,sex_priv,0.000,0.000,0.185,-0.392,0.258,0.044,0.248,-0.379,-0.020,-0.083,-0.074,0.784,0.312,0.069,-0.258,0.392
9,COMPAS_Without_Sensitive_Attributes,LogisticRegression,sex_dis,0.077,0.067,0.236,-0.478,0.316,0.111,0.274,-0.294,-0.015,-0.105,-0.012,0.801,0.397,0.087,-0.316,0.478
